### Presimulation Notebook # 1 - Flood Risk Overlay


This notebook performs a flood risk assessment by combining hazard, exposure, and vulnerability data

In [8]:
# Import live code changes in
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from sovereign.flood import simple_risk_overlay, flopros_risk_overlay
from sovereign.utils import sum_rasters

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### 1. Set filepaths

In [9]:
root = Path.cwd().parent.parent # find project root
flood_dir = os.path.join(root, 'inputs', 'flood', 'maps')
adaptation_dir = os.path.join(root, 'inputs', 'flood', 'adapation')
exposure_dir = os.path.join(root, 'outputs', 'exposure')
vulnerability_path = os.path.join(root, 'inputs', 'flood', 'vulnerability', 'jrc_depth_damage.csv')
# Dictionary of flood maps
flood_dic = {10: 'THA_jrc-flood_RP10.tif',
            20: 'THA_jrc-flood_RP20.tif',
            50: 'THA_jrc-flood_RP50.tif',
            75: 'THA_jrc-flood_RP75.tif',
            100: 'THA_jrc-flood_RP100.tif',
            200: 'THA_jrc-flood_RP200.tif',
            500: 'THA_jrc-flood_RP500.tif'}
# Capital stock paths
priv_inf = os.path.join(exposure_dir, 'inf_priv_capstock.tif')
priv_res = os.path.join(exposure_dir, 'res_priv_capstock.tif')
priv_nres = os.path.join(exposure_dir, 'nres_priv_capstock.tif')
pub_inf = os.path.join(exposure_dir, 'inf_pub_capstock.tif')
pub_nres = os.path.join(exposure_dir, 'nres_pub_capstock.tif')
# GVA paths
agr_gva = os.path.join(exposure_dir, 'agr_GVA.tif')
man_gva = os.path.join(exposure_dir, 'man_GVA.tif')
ser_gva = os.path.join(exposure_dir, 'ser_GVA.tif')
# Flood risk map folder
risk_results = os.path.join(root, 'outputs', 'flood', 'risk', 'maps')

##### 2. Extract vulnerability curves

In [10]:
vuln_df = pd.read_csv(vulnerability_path)
v_heights = vuln_df['flood_depth'].to_list()
v_res = vuln_df['asia_residential'].to_list()
v_com = vuln_df['asia_commercial'].to_list()
v_ind = vuln_df['asia_industrial'].to_list()
v_agr = vuln_df['asia_agriculture'].to_list()
v_inf = vuln_df['asia_infrastructure'].to_list()

##### 3. Run risk analysis (writing flood risk rasters for each sector & flood RP combination)

In [11]:
# Check to see if results directory exists
if not os.path.exists(risk_results):
    os.makedirs(risk_results)

def run_if_missing(output_path, func, *args):
    if os.path.exists(output_path):
        pbar.update(1)
        return
    func(*args)
    pbar.update(1)

rps = list(flood_dic)
total_ops = len(rps) * 10 # for each return period we are going to produce 10 rasters

with tqdm(total=total_ops, desc="Running risk overlays", unit="raster") as pbar:

    for RP in rps:

        flood_path = os.path.join(flood_dir, flood_dic[RP])

        priv_inf_risk = os.path.join(risk_results, f'JRC_{RP}_priv_inf_cap_damages.tif')
        priv_res_risk = os.path.join(risk_results, f'JRC_{RP}_priv_res_cap_damages.tif')
        priv_nres_risk = os.path.join(risk_results, f'JRC_{RP}_priv_nres_cap_damages.tif')
        pub_inf_risk  = os.path.join(risk_results, f'JRC_{RP}_pub_inf_cap_damages.tif')
        pub_nres_risk = os.path.join(risk_results, f'JRC_{RP}_pub_nres_cap_damages.tif')
        agr_gva_risk  = os.path.join(risk_results, f'JRC_{RP}_agr_gva_losses.tif')
        man_gva_risk  = os.path.join(risk_results, f'JRC_{RP}_man_gva_losses.tif')
        ser_gva_risk  = os.path.join(risk_results, f'JRC_{RP}_ser_gva_losses.tif')

        # ---- Overlays ----
        run_if_missing(priv_inf_risk, simple_risk_overlay,
                       flood_path, priv_inf, priv_inf_risk, [v_heights, v_inf])

        run_if_missing(priv_res_risk, simple_risk_overlay,
                       flood_path, priv_res, priv_res_risk, [v_heights, v_res])

        run_if_missing(priv_nres_risk, simple_risk_overlay,
                       flood_path, priv_nres, priv_nres_risk, [v_heights, v_com])

        run_if_missing(pub_inf_risk, simple_risk_overlay,
                       flood_path, pub_inf, pub_inf_risk, [v_heights, v_inf])

        run_if_missing(pub_nres_risk, simple_risk_overlay,
                       flood_path, pub_nres, pub_nres_risk, [v_heights, v_com])

        run_if_missing(agr_gva_risk, simple_risk_overlay,
                       flood_path, agr_gva, agr_gva_risk, [v_heights, v_agr])

        run_if_missing(man_gva_risk, simple_risk_overlay,
                       flood_path, man_gva, man_gva_risk, [v_heights, v_ind])

        run_if_missing(ser_gva_risk, simple_risk_overlay,
                       flood_path, ser_gva, ser_gva_risk, [v_heights, v_com])

        # ---- Sums ----

        priv_sum_path = os.path.join(risk_results, f'JRC_{RP}_priv_cap_damages.tif')
        pub_sum_path  = os.path.join(risk_results, f'JRC_{RP}_pub_cap_damages.tif')

        if os.path.exists(priv_sum_path):
            pbar.update(1)
        else:
            sum_rasters(
                [priv_inf_risk, priv_res_risk, priv_nres_risk],
                priv_sum_path
            )
            pbar.update(1)

        if os.path.exists(pub_sum_path):
            pbar.update(1)
        else:
            sum_rasters(
                [pub_inf_risk, pub_nres_risk],
                pub_sum_path
            )
            pbar.update(1)

Running risk overlays: 100%|█████████████████████████████████████████████████████| 70/70 [1:00:00<00:00, 51.44s/raster]
